In [ ]:
import pandas as pd

# อ่านข้อมูลหลัก (ที่มี Geography Name)
df = pd.read_csv('select-data.csv')

# อ่านไฟล์อ้างอิง census tract → city
df_tract = pd.read_csv('census-tracts.csv')

df.head(), df_tract.head()

In [ ]:
import numpy as np

def norm_tract_code(x):
    if pd.isna(x):
        return np.nan
    x = float(x)
    s = f"{x:.2f}"        # แปลงเป็น string แบบมีทศนิยม 2 ตำแหน่ง เช่น 2.01, 1.00
    if s.endswith(".00"): # ถ้าเป็นเลขเต็ม เช่น 1.00 → เอาเป็น '1'
        s = s[:-3]
    else:
        s = s.rstrip('0').rstrip('.')  # ตัดศูนย์เกิน ๆ ด้านท้าย
    return s

df_tract['tract_str'] = df_tract['tract_code'].apply(norm_tract_code)
df_tract[['tract_code', 'tract_str', 'city']].head()

In [ ]:
def extract_tract(name):
    if isinstance(name, str) and name.startswith("Census Tract"):
        # ตัดเอาเฉพาะตัวเลขด้านหลังคำว่า "Census Tract"
        return name.split("Census Tract", 1)[1].strip()
    return np.nan

df['tract_str'] = df['Geography Name'].apply(extract_tract)
df[['Geography Name', 'tract_str']].head(20)

In [ ]:
df_merged = df.merge(
    df_tract[['tract_str', 'city']],
    how='left',
    on='tract_str'
)

df_merged.head()

In [ ]:
mask = df_merged['Geography Name'].str.startswith('Census Tract')

# เปลี่ยน Geography Name เฉพาะแถวที่เป็น Census Tract ให้เป็นชื่อเมือง
df_merged.loc[mask, 'Geography Name'] = df_merged.loc[mask, 'city']

df_merged[['Year', 'Geography Name', 'Poverty Estimate']].head(20)

In [ ]:
df_merged.to_csv('merge-column.csv', index=False)